# Model Selection and Tuning

This file is concerned with trying different ML algorithms for classification, focusing exclusively on predictive power and starting from the simplest model to the most complicated. We will use the processed data created from the data_preparation.ipynb file, which is a csv named 'aggregated_train_data.csv'.

The objective of the models is to accurately predict if a client with certain characteristics will default on a loan when asking for it. As we do not know the associated costs with a client not paying back nor the benefits of a client repaying, we will focus on Recall as the primary performance metric. We, however, want to create a model that will not reject that many clients who would repay and will therefore also pay attention to Area Under Curve and Accuracy, hoping that the model will be powerful in identifying defaulting loans while still being balanced.

Every model will be trained, tuned and cross-validated using the sci-kit learn library.

In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

jobs = os.cpu_count()-2 ## A lot of power.

In [2]:
df = pd.read_csv('aggregated_train_data.csv', index_col= 0)
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,CC_SK_DPD_MEAN,CC_SK_DPD_DEF_MEAN,CC_NAME_CONTRACT_STATUS_Active_MEAN,CC_NAME_CONTRACT_STATUS_Approved_MEAN,CC_NAME_CONTRACT_STATUS_Completed_MEAN,CC_NAME_CONTRACT_STATUS_Demand_MEAN,CC_NAME_CONTRACT_STATUS_Refused_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_nan_MEAN
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.rename(columns = {col: col.lower() for col in df.columns.values}, inplace = True)
X = df.drop(columns=['target','sk_id_curr'])
y = df.target.copy()

performance_metrics = {}

In [4]:
y.value_counts()

0    282686
1     24825
Name: target, dtype: int64

We note that we will be dealing with imbalanced data. The negative (non-default in this case) are above 90% of the observations while the positive (client defaulting on a loan) are less than 10% of the cases.

### First Model: **Logistic Regression**

For this model we will first fit a logistic regression to all the parameters, and take the mean of the AUC score from a 5 fold cross validation. We will then run a logistic regression regularized by Lasso to try and do feature extraction. We will finally compute the regression without the penalty but using only the parameters extracted from the constrained regression.

In [5]:
from sklearn.linear_model import SGDClassifier

In [6]:
## Logistic Regression and other models do not accept NaN values. Will use sklearn's preprocessing to impute mean where necessary.

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

X.replace([np.inf, -np.inf], np.nan, inplace=True)
transformations = Pipeline([('impute', SimpleImputer(strategy= 'mean')), ('scale', StandardScaler())])
transformed_data = transformations.fit_transform(X)

All variables

In [7]:
start = time.time()
logistic_regression_full = SGDClassifier(loss = 'log_loss', l1_ratio= 1, n_jobs= jobs, random_state= 0, early_stopping= True, class_weight='balanced')
res = cross_validate(logistic_regression_full, transformed_data,y, cv = 5, scoring=['recall','roc_auc','accuracy'])
end = time.time()
performance_metrics['logistic_regression_full'] = {'recall' : np.mean(res['test_recall']),'roc_auc': np.mean(res['test_roc_auc']),'accuracy':np.mean(res['test_accuracy']),'runtime': end-start}

Regularized Logistic regression

In [8]:
## Model tuning

##param_grid = [{'alpha' : [0.0001,0.0005,0.001]}]
##logistic_regression_l1 = SGDClassifier(loss = 'log_loss', penalty = 'l1', l1_ratio = 1, n_jobs= jobs, random_state= 0, early_stopping= True, class_weight='balanced')
##grid_search = GridSearchCV(logistic_regression_l1, param_grid, scoring=['recall','roc_auc','accuracy'], refit = 'recall')
##grid_search.fit(transformed_data,y)
#grid_search.best_params_

In [9]:
#grid_search.best_score_

In [10]:
start = time.time()
logistic_regression_l1 = SGDClassifier(loss = 'log_loss', alpha= 0.0001, penalty = 'l1', l1_ratio= 1, n_jobs= jobs, random_state= 0, early_stopping= True, class_weight='balanced')
res = cross_validate(logistic_regression_l1, transformed_data,y, cv = 5, scoring=['recall','roc_auc','accuracy'])
end = time.time()
performance_metrics['logistic_regression_l1'] = {'recall' : np.mean(res['test_recall']),'roc_auc': np.mean(res['test_roc_auc']),'accuracy':np.mean(res['test_accuracy']),'runtime': end-start}

### Second Model: **Support Vector Machines**

In [11]:
from sklearn.linear_model import SGDClassifier

In [12]:
## Model tuning

##svc = SGDClassifier(loss = 'hinge', random_state = 0, max_iter = 10000, early_stopping = True, class_weight='balanced')
##param_grid = [{'alpha': [0.01,0.05,0.1]}]
##grid_search = GridSearchCV(svc, param_grid, n_jobs = jobs, cv = 5, scoring = 'recall')
##grid_search.fit(transformed_data,y)
##grid_search.best_params_

In [13]:
##grid_search.best_score_

In [14]:
start = time.time()
svc = SGDClassifier(loss = 'hinge', alpha= 0.05,random_state= 0, max_iter = 10000, early_stopping= True, class_weight='balanced')
res = cross_validate(svc, transformed_data,y, cv = 5, scoring=['recall','roc_auc','accuracy'])
end = time.time()
performance_metrics['SVC'] = {'recall' : np.mean(res['test_recall']),'roc_auc': np.mean(res['test_roc_auc']),'accuracy':np.mean(res['test_accuracy']),'runtime': end-start}

### Third Model: **Decision Tree**

In [15]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
## Model tuning

##clf = DecisionTreeClassifier(random_state = 0, class_weight='balanced')
##param_grid = [{'max_depth':[11,13,15]}]
##grid_search = GridSearchCV(clf, param_grid, n_jobs = jobs, cv = 5, scoring = 'recall')
##grid_search.fit(transformed_data,y)
##grid_search.best_params_

In [17]:
##grid_search.best_score_

In [18]:
start = time.time()
dec_tree = DecisionTreeClassifier(max_depth = 11, random_state = 0, class_weight='balanced')
res = cross_validate(dec_tree, transformed_data,y, cv = 5, n_jobs = jobs, scoring=['recall','roc_auc','accuracy'])
end = time.time()
performance_metrics['decision_tree'] = {'recall' : np.mean(res['test_recall']),'roc_auc': np.mean(res['test_roc_auc']),'accuracy':np.mean(res['test_accuracy']),'runtime': end-start}

### Fourth Model: **Random Forest**

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
## Model tuning

##clf = RandomForestClassifier(random_state = 0, class_weight='balanced')
##param_grid = [{'max_depth':[11,13,15], 'min_samples_leaf': [7,9,11], 'n_estimators': 200}]
##grid_search = GridSearchCV(clf, param_grid, n_jobs = jobs, cv = 5, scoring = 'recall')
##grid_search.fit(transformed_data,y)
##grid_search.best_params_

In [21]:
start = time.time()
clf = RandomForestClassifier(n_estimators = 1000, max_depth = 12, min_samples_leaf = 7, n_jobs = jobs,random_state= 0, class_weight='balanced')
res = cross_validate(clf, transformed_data,y, scoring=['recall','roc_auc','accuracy'])
end = time.time()
performance_metrics['random_forest'] = {'recall' : np.mean(res['test_recall']),'roc_auc': np.mean(res['test_roc_auc']),'accuracy':np.mean(res['test_accuracy']),'runtime': end-start}

### Fifth Model: **XGBoost**

Tuned and executed here: [XGBoost](https://github.com/roberto-andrade22/loan_default_classification/blob/main/XGBoost.ipynb)

### Comparing model performance with runtime:

All of the metrics were obtained with kfold cross-validation, so that they are more robust and closer to the actual test scores.

In [22]:
results = pd.DataFrame(performance_metrics).transpose()
results.sort_values(by = ['recall'], ascending= False)

,recall,roc_auc,accuracy,runtime
SVC,0.696032,0.767872,0.705386,15.238288
decision_tree,0.629849,0.680714,0.665443,59.017263
logistic_regression_l1,0.628399,0.655834,0.618775,32.304768
logistic_regression_full,0.590211,0.643979,0.621308,16.187219
random_forest,0.481088,0.750574,0.810787,2158.852777


In [23]:
results.to_csv('performance_metrics.csv')

Further analysis in the XGBoost notebook.